In [46]:
import io
import numpy as np
import pprint
import pycurl

In [19]:
ip = 'frontend.hpc4ai.unito.it:4567'
pandora_url = '/pandora_console'
apipass = '12345678'
user = 'stefaniak'
password = 'simplepass'
op = 'get'


url = "https://" + ip  + pandora_url + "/include/api.php"

url += "?"
url += "apipass=" + apipass
url += "&user="  + user
url += "&pass=" + password
url += "&op=" + op

get_url = url # save url with op=get for later

op2 = 'all_agents'
return_type = 'csv'
url += "&op2=" + op2
url += "&return_type=" + return_type

In [20]:
print(url)

https://frontend.hpc4ai.unito.it:4567/pandora_console/include/api.php?apipass=12345678&user=stefaniak&pass=simplepass&op=get&op2=all_agents&return_type=csv


In [21]:
def url_out(url):
    buf = io.BytesIO()

    c = pycurl.Curl()
    c.setopt(c.URL, url)
    c.setopt(c.WRITEFUNCTION, buf.write)
    c.perform()

    output = buf.getvalue().decode('UTF-8')
    buf.close()
    
    return output

In [22]:
output = url_out(url)

In [23]:
lines = output.split("\n")
agents = []
for line in lines:
    if not line:
        continue
    
    fields = line.split(";")
    agent = {}
    agent['id_agent'] = fields[0]
    agent['name'] = fields[1]
    agent['ip'] = fields[2]
    agent['description'] = fields[3]
    agent['os_name'] = fields[4]
    agent['url_address'] = fields[5]

    agents.append(agent)

In [24]:
print(agents)

[{'id_agent': '56', 'name': 'broadwell-000', 'ip': '192.168.100.155', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '57', 'name': 'broadwell-001', 'ip': '192.168.100.156', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '58', 'name': 'broadwell-002', 'ip': '192.168.100.22', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '59', 'name': 'broadwell-003', 'ip': '192.168.100.23', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '60', 'name': 'broadwell-004', 'ip': '192.168.100.24', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '61', 'name': 'broadwell-005', 'ip': '192.168.100.25', 'description': 'Created by pandorafms.maas', 'os_name': 'Linux', 'url_address': ''}, {'id_agent': '62', 'name': 'broadwell-006', 'ip': '192.168.100.26', 'descriptio

In [28]:
id_agent = '59'

op2 = 'agent_modules'
return_type = 'csv'
other = id_agent
other_mode = 'url_encode_separator_|'

url = get_url
url += "&op2=" + op2
url += "&return_type=" + return_type
url += "&other=" + other
url += "&other_mode=" + other_mode

output = url_out(url)

In [32]:
modules = []
for agent in agents:
    op2 = 'agent_modules'
    return_type = 'csv'
    other = agent['id_agent']
    other_mode = 'url_encode_separator_|'

    url = get_url
    url += "&op2=" + op2
    url += "&return_type=" + return_type
    url += "&other=" + other
    url += "&other_mode=" + other_mode
    
    output = url_out(url)
    
    lines = output.split("\n")
    for line in lines:
        if not line:
            continue
    
        fields = line.split(";")
        module = {}
        module['id_agent'] = fields[0]
        module['id_module'] = fields[1]
        module['name'] = fields[2]

        modules.append(module)  

In [40]:
op2 = 'module_data'
id_module = modules[1]['id_module']


url = get_url
url += "&op2=" + op2
url += "&id=" + id_module

output = url_out(url)

In [53]:
lines = output.split("\n")
values = np.empty(len(lines), dtype = float)

for i, line in enumerate(lines[1:]):
    if not line:
        continue
    
    fields = line.split(";")
    values[i] = float(fields[1])


In [54]:
values

array([2.00000000e-002, 2.00000000e-002, 0.00000000e+000, ...,
       0.00000000e+000, 6.91351208e-310, 6.91351208e-310])